## Function to modify the distance between plates

In [46]:
import os
import re

def modify_plates_distance(geometry, new_gap, name):
    # Open the geometry.geo file to read the lines
    with open(str(geometry), "r") as f:
        lines = f.readlines()
        
    new_lines = []
    for line in lines:

        if 'Point(1) =' in line:
            # Split the line by commas, remove curly braces, and strip whitespace
            parts = line.split('{')[1].split('}')[0].split(',')
            # Update the second value
            parts[1] = str(new_gap / 2)
            # Recreate the line
            new_line = f"Point(1) = {{{','.join(parts)}}};\n"
            new_lines.append(new_line)

        elif 'Rotate' in line:
            # Extract the part after 'Rotate {{0, 0, 1},'
            rest = line.split("Rotate {{0, 0, 1},", 1)[1]

            # Find end of coordinate block
            coord_end = rest.find("}")
            coords_str = rest[:coord_end + 1].strip()  # includes closing brace

            # Get the angle part (right after coords)
            angle_str = rest[coord_end + 2:].split("}")[0].strip()

            # Parse and update y value (coords[1])
            coords = coords_str.strip("{}").split(',')
            coords = [c.strip() for c in coords]
            coords[1] = str(new_gap / 2)

            # Reconstruct the line
            new_line = f"Rotate {{ {{0, 0, 1}}, {{{', '.join(coords)}}}, {angle_str} }} {{\n"
            new_lines.append(new_line)
            
        elif 'Point(2) =' in line:
            # Split the line by commas, remove curly braces, and strip whitespace
            parts = line.split('{')[1].split('}')[0].split(',')
            # Update the second value
            parts[1] = str(new_gap / 2)
            # Recreate the line
            new_line = f"Point(2) = {{{','.join(parts)}}};\n"
            new_lines.append(new_line)
        elif 'Point(3) =' in line:
            # Split the line by commas, remove curly braces, and strip whitespace
            parts = line.split('{')[1].split('}')[0].split(',')
            # Update the second value
            parts[1] = str(new_gap / 2 + 4)
            # Recreate the line
            new_line = f"Point(3) = {{{','.join(parts)}}};\n"
            new_lines.append(new_line)
        elif 'Point(4) =' in line:
            # Split the line by commas, remove curly braces, and strip whitespace
            parts = line.split('{')[1].split('}')[0].split(',')
            # Update the second value
            parts[1] = str(new_gap / 2 + 4)
            # Recreate the line
            new_line = f"Point(4) = {{{','.join(parts)}}};\n"
            new_lines.append(new_line)
        elif 'Point(5) =' in line:
            # Split the line by commas, remove curly braces, and strip whitespace
            parts = line.split('{')[1].split('}')[0].split(',')
            # Update the second value
            parts[1] = str(-new_gap / 2)
            # Recreate the line
            new_line = f"Point(5) = {{{','.join(parts)}}};\n"
            new_lines.append(new_line)
        elif 'Point(6) =' in line:
            # Split the line by commas, remove curly braces, and strip whitespace
            parts = line.split('{')[1].split('}')[0].split(',')
            # Update the second value
            parts[1] = str(-new_gap / 2)
            # Recreate the line
            new_line = f"Point(6) = {{{','.join(parts)}}};\n"
            new_lines.append(new_line)
        elif 'Point(7) =' in line:
            # Split the line by commas, remove curly braces, and strip whitespace
            parts = line.split('{')[1].split('}')[0].split(',')
            # Update the second value
            parts[1] = str(-new_gap / 2 - 4)
            # Recreate the line
            new_line = f"Point(7) = {{{','.join(parts)}}};\n"
            new_lines.append(new_line)
        elif 'Point(8) =' in line:
            # Split the line by commas, remove curly braces, and strip whitespace
            parts = line.split('{')[1].split('}')[0].split(',')
            # Update the second value
            parts[1] = str(-new_gap / 2 - 4)
            # Recreate the line
            new_line = f"Point(8) = {{{','.join(parts)}}};\n"
            new_lines.append(new_line)
        else:
            # If the line doesn't match any of the points, keep it unchanged
            new_lines.append(line)
    
    # Define the directory and file name for saving the new geometry
    directory = "meshes"
    file_name = str(name) + ".geo"
    
    # Create the directory if it doesn't exist
    os.makedirs(directory, exist_ok=True)
    
    # Define the full file path
    file_path = os.path.join(directory, file_name)
    
    # Write the modified lines to the new geometry file
    with open(file_path, "w") as f:
        f.writelines(new_lines)

    print(f"Geometry updated with a gap of {new_gap}. Saved to {file_path}")

# Function to modify overetch of plates

In [47]:
import os
import re

def modify_plates_overetch(geometry, overetch, name):
    # Open the geometry.geo file to read the lines
    with open(str(geometry), "r") as f:
        lines = f.readlines()
        
    new_lines = []
    for line in lines:

        # first rectangle
        if 'Point(1) =' in line:
            parts = line.split('{')[1].split('}')[0].split(',')
            parts[0] = f"{float(parts[0].strip()) + overetch}"
            new_line = f"Point(1) = {{{', '.join(part.strip() for part in parts)}}};\n"
            new_lines.append(new_line)
        elif 'Point(2) =' in line:
            parts = line.split('{')[1].split('}')[0].split(',')
            parts[0] = f"{float(parts[0].strip()) - overetch}"
            new_line = f"Point(2) = {{{', '.join(part.strip() for part in parts)}}};\n"
            new_lines.append(new_line)
        elif 'Point(3) =' in line:
            parts = line.split('{')[1].split('}')[0].split(',')
            parts[0] = f"{float(parts[0].strip()) - overetch}"
            parts[1] = f"{float(parts[1].strip()) - 2 * overetch}"
            new_line = f"Point(3) = {{{', '.join(part.strip() for part in parts)}}};\n"
            new_lines.append(new_line)
        elif 'Point(4) =' in line:
            parts = line.split('{')[1].split('}')[0].split(',')
            parts[0] = f"{float(parts[0].strip()) + overetch}"
            parts[1] = f"{float(parts[1].strip()) - 2 * overetch}"
            new_line = f"Point(4) = {{{', '.join(part.strip() for part in parts)}}};\n"
            new_lines.append(new_line)

        # second rectangle
        elif 'Point(5) =' in line:
            parts = line.split('{')[1].split('}')[0].split(',')
            parts[0] = f"{float(parts[0].strip()) + overetch}"
            new_line = f"Point(5) = {{{', '.join(part.strip() for part in parts)}}};\n"
            new_lines.append(new_line)
        elif 'Point(6) =' in line:
            parts = line.split('{')[1].split('}')[0].split(',')
            parts[0] = f"{float(parts[0].strip()) - overetch}"
            new_line = f"Point(6) = {{{', '.join(part.strip() for part in parts)}}};\n"
            new_lines.append(new_line)
        elif 'Point(7) =' in line:
            parts = line.split('{')[1].split('}')[0].split(',')
            parts[0] = f"{float(parts[0].strip()) - overetch}"
            parts[1] = f"{float(parts[1].strip()) + 2 * overetch}"
            new_line = f"Point(7) = {{{', '.join(part.strip() for part in parts)}}};\n"
            new_lines.append(new_line)
        elif 'Point(8) =' in line:
            parts = line.split('{')[1].split('}')[0].split(',')
            parts[0] = f"{float(parts[0].strip()) + overetch}"
            parts[1] = f"{float(parts[1].strip()) + 2 * overetch}"
            new_line = f"Point(8) = {{{', '.join(part.strip() for part in parts)}}};\n"
            new_lines.append(new_line)

        # ignore other lines
        else:
            new_lines.append(line)

    
    # Define the directory and file name for saving the new geometry
    directory = "meshes"
    file_name = str(name) + ".geo"
    
    # Create the directory if it doesn't exist
    os.makedirs(directory, exist_ok=True)
    
    # Define the full file path
    file_path = os.path.join(directory, file_name)
    
    # Write the modified lines to the new geometry file
    with open(file_path, "w") as f:
        f.writelines(new_lines)

    print(f"Geometry updated with an overetch of {overetch}. Saved to {file_path}")


## Function to rotate a rectangle

In [48]:
import re
import os
from math import pi

def rotate_upper_plate(geometry, new_angle, name):
    with open(str(geometry), "r") as f:
        lines = f.readlines()

    new_lines = []
    for line in lines:
        if 'Rotate' in line:
            # Extract text after 'Rotate {{0, 0, 1},'
            rest = line.split("Rotate { {0, 0, 1},", 1)[1]
            
            # Split at the last '}' to isolate the coordinates
            coord_end = rest.rfind('}')
            coords_str = rest[:coord_end+1].strip()  # includes closing }

            # Rebuild the line with the new angle in the correct position
            # Remove the last value (angle) from coords_str
            coords_str = ', '.join(coords_str.strip('{}').split(',')[:-1])
            new_line = f"Rotate {{ {{0, 0, 1}}, {{{coords_str}, {round(new_angle*pi/180, 3)} }} {{\n"
            new_lines.append(new_line)
        else:
            new_lines.append(line)

    # Define the directory and file name
        directory = "meshes"
    file_name = str(name) + ".geo"

    # Create the directory if it doesn't exist
    os.makedirs(directory, exist_ok=True)
    # Create the file in the directory
    file_path = os.path.join(directory, file_name)
    with open(file_path, "w") as f:
        f.writelines(new_lines)

    print(f"Geometry updated with a rotation of {new_angle} degrees. Saved to {file_path}")



## Mesh generation

In [ ]:

import gmsh
import os

def generate_mesh_from_geo(geo_path):

    # Initialize Gmsh
    gmsh.initialize()

    # Optional: Hide Gmsh messages
    gmsh.option.setNumber("General.Terminal", 1)

    # Load the .geo file
    gmsh.open(geo_path)

    # Generate 2D or 3D mesh depending on your .geo setup
    gmsh.model.mesh.generate(2)

    # Write the mesh to a .msh file
    msh_path = os.path.splitext(geo_path)[0] + ".msh"
    gmsh.write(msh_path)

    # Finalize Gmsh
    gmsh.finalize()

generate_mesh_from_geo("geometry.geo")

Info    : I'm busy! Ask me that later...
Info    : I'm busy! Ask me that later...
Info    : Writing 'meshes/rotated_geometry.msh'...
Info    : Done writing 'meshes/rotated_geometry.msh'


## Generation of different geometries and correspoding meshes

In [53]:
import numpy as np

import shutil

def reset_environment():
    meshes_folder = "meshes"
    parameters_file = "data/parameters.csv"
    # Empty meshes folder
    if os.path.exists(meshes_folder):
        shutil.rmtree(meshes_folder)
        os.makedirs(meshes_folder)

    # Ensure the parameters.csv file is empty
    if os.path.exists(parameters_file):
        with open(parameters_file, "w") as csv_file:
            csv_file.write("ID,Overetch,Distance,Angle\n")
    else:
        os.makedirs("data", exist_ok=True)
        with open(parameters_file, "w") as csv_file:
            csv_file.write("ID,Overetch,Distance,Angle\n")

def generate_meshes():
    overetches = np.linspace(0.1, 0.6, 10)
    distances = np.linspace(1.5, 2.5, 10)
    angles = np.linspace(1, -1, 10)

    # Ensure the parameters.csv file is empty before writing
    with open("data/parameters.csv", "w") as csv_file:
        csv_file.write("ID,Overetch,Distance,Angle\n")
        csv_file.truncate()
    
    reset_environment()

    j = 1
    for o in overetches:
        for d in distances:
            for a in angles:
                modify_plates_distance("geometry.geo", d, j)
                modify_plates_overetch("meshes/" + str(j) + ".geo", o, j)
                rotate_upper_plate("meshes/" + str(j) + ".geo", a, j)
                j += 1
                with open("data/parameters.csv", "a") as csv_file:
                    csv_file.write(f"{j},{o},{d},{a}\n")

if __name__ == "__main__":
    generate_meshes()
    print("Mesh files generated successfully.")

Geometry updated with a gap of 1.5. Saved to meshes/1.geo
Geometry updated with an overetch of 0.1. Saved to meshes/1.geo
Geometry updated with a rotation of 1.0 degrees. Saved to meshes/1.geo
Geometry updated with a gap of 1.5. Saved to meshes/2.geo
Geometry updated with an overetch of 0.1. Saved to meshes/2.geo
Geometry updated with a rotation of 0.7777777777777778 degrees. Saved to meshes/2.geo
Geometry updated with a gap of 1.5. Saved to meshes/3.geo
Geometry updated with an overetch of 0.1. Saved to meshes/3.geo
Geometry updated with a rotation of 0.5555555555555556 degrees. Saved to meshes/3.geo
Geometry updated with a gap of 1.5. Saved to meshes/4.geo
Geometry updated with an overetch of 0.1. Saved to meshes/4.geo
Geometry updated with a rotation of 0.33333333333333337 degrees. Saved to meshes/4.geo
Geometry updated with a gap of 1.5. Saved to meshes/5.geo
Geometry updated with an overetch of 0.1. Saved to meshes/5.geo
Geometry updated with a rotation of 0.11111111111111116 degr

In [55]:
import multiprocessing

def generate_mesh(i):
    # Generate the mesh for each geometry
    geo_path = f"meshes/{i}.geo"
    generate_mesh_from_geo(geo_path)

r = range(1, 1001)
with multiprocessing.Pool() as pool:
    pool.map(generate_mesh, r)

Info    : Reading 'meshes/1.geo'...
Info    : Reading 'meshes/22.geo'...
Info    : Reading 'meshes/43.geo'...
Info    : Reading 'meshes/64.geo'...
Info    : Reading 'meshes/85.geo'...
Info    : Reading 'meshes/106.geo'...
Info    : Reading 'meshes/127.geo'...
Info    : Reading 'meshes/169.geo'...
Info    : Reading 'meshes/148.geo'...
Info    : Reading 'meshes/190.geo'...
Info    : Reading 'meshes/211.geo'...
Info    : Reading 'meshes/232.geo'...
Info    : Meshing 1D...ence - Making faces                                                                                              : [ 80%] Difference - Making faces                                                                                
Info    : Meshing 1D...
Info    : Meshing 1D...
Info    : Meshing 1D...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)les                                                                                           
Info    : [  0%] Meshing curve 1 (Line)
Info    : [  0%] Meshing curv

KeyboardInterrupt: 